Use meandering triangles to compute isolines

From https://blog.bruce-hill.com/code?f=meandering-triangles/meandering_triangles.py

In [1]:
import math
import collections
import numpy as np
import pandas as pd
import altair as alt

Triangle = collections.namedtuple("Triangle", "v1 v2 v3")
Edge = collections.namedtuple("Edge", "e1 e2")

In [2]:
def find_isolines(x, y, z, elevation):
    '''
    Return a list of contour lines that have a constant elevation near the specified
    value. Lines will be truncated to the sepcified x/y range, and the elevation
    function will be sampled at even intervals determined by the spacing parameter.
    '''
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)
    
    elevation_data = {}
    triangles = []
    for ix in range(0, len(x)-1):
        for iy in range(0, len(y)-1):
            triangles.append(Triangle((x[ix], y[iy]), (x[ix+1], y[iy]), (x[ix], y[iy+1])))
            triangles.append(Triangle((x[ix+1], y[iy]), (x[ix], y[iy+1]), (x[ix+1],y[iy+1])))
            
    try:
        for ix in range(0, len(x)):
            for iy in range(0, len(y)):
                elevation_data[(x[ix], y[iy])] = z[ix][iy]
    except IndexError as e:
        print('len(x)=',len(x), 'ix=', ix, 'len(y)=', len(y), 'iy=', iy, 'z.shape=', z.shape, e)
        
    #for iz in elevation_data.keys():
    #    print(iz)

    contour_segments = []
    for triangle in triangles:
        below = [v for v in triangle if elevation_data[v] < elevation]
        above = [v for v in triangle if elevation_data[v] >= elevation]
        # All above or all below means no contour line here
        if len(below) == 0 or len(above) == 0:
            continue
        # We have a contour line, let's find it
        minority = above if len(above) < len(below) else below
        majority = above if len(above) > len(below) else below

        contour_points = []
        crossed_edges = (Edge(minority[0], majority[0]), Edge(minority[0], majority[1]))
        for triangle_edge in crossed_edges:
            # how_far is a number between 0 and 1 indicating what percent of the way
            # along the edge (e1,e2) the crossing point is
            how_far = ((elevation - elevation_data[triangle_edge.e2])
                     / (elevation_data[triangle_edge.e1] - elevation_data[triangle_edge.e2]))
            crossing_point = (
                how_far * triangle_edge.e1[0] + (1-how_far) * triangle_edge.e2[0],
                how_far * triangle_edge.e1[1] + (1-how_far) * triangle_edge.e2[1])
            contour_points.append(crossing_point)
        contour_segments.append(Edge(contour_points[0], contour_points[1]))

    unused_segments = set(contour_segments)
    segments_by_point = collections.defaultdict(set)
    for segment in contour_segments:
        segments_by_point[segment.e1].add(segment)
        segments_by_point[segment.e2].add(segment)

    contour_lines = []
    while unused_segments:
        # Start with a random segment
        line = collections.deque(unused_segments.pop())
        while True:
            tail_candidates = segments_by_point[line[-1]].intersection(unused_segments)
            if tail_candidates:
                tail = tail_candidates.pop()
                line.append(tail.e1 if tail.e2 == line[-1] else tail.e2)
                unused_segments.remove(tail)
            head_candidates = segments_by_point[line[0]].intersection(unused_segments)
            if head_candidates:
                head = head_candidates.pop()
                line.appendleft(head.e1 if head.e2 == line[0] else head.e2)
                unused_segments.remove(head)
            if not tail_candidates and not head_candidates:
                # There are no more segments touching this line, so we're done with it.
                contour_lines.append(list(line))
                break

    return contour_lines

In [3]:
xr = np.linspace(-5, 5, 50)
yr = np.linspace(-5, 5, 50)
x, y = np.meshgrid(xr, yr)
z = x ** 2 + y ** 2
print(len(xr), len(yr), z.shape)

50 50 (50, 50)


In [4]:
source = pd.DataFrame({'x': x.ravel(),
                     'y': y.ravel(),
                     'z': z.ravel()})

contour = alt.Chart(source).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='z:Q'
).properties(
    width=200,
    height=200
)


In [5]:
def isolines2lines(cl, minorder):
    # add a None/null point after each line to have altair jump to the next one
    cl_x = [ [cl[i][j][0] for j in range(0, len(cl[i]))]  + [None] for i in range(0, len(cl))]
    cl_x1 = [cl_x[i][j] for i in range(0, len(cl_x)) for j in range(0, len(cl_x[i]))]
    cl_y = [ [cl[i][j][1] for j in range(0, len(cl[i]))] + [None] for i in range(0, len(cl))]
    cl_y1 = [cl_y[i][j] for i in range(0, len(cl_y)) for j in range(0, len(cl_y[i]))]
    # order is required because by default, altair will plot according to x axis
    order = [minorder+i for i in range(0, len(cl_x1))]
    return cl_x1, cl_y1, order

def isolines(x, y, z, values):
    dfs = []
    # use minorder to differentiate lines when dataframes are concatenated
    minorder = 0
    for v in values:
        cl = find_isolines(x, y, z, v)
        cl_x, cl_y, cl_order = isolines2lines(cl, minorder)
        minorder += len(cl_x)
        # val is to differentiate between isolines, also used as a legend
        cl_val = [v for i in range(0, len(cl_x))]
        dfs.append(pd.DataFrame({'x': cl_x, 'y': cl_y, 'order': cl_order, 'iso': cl_val}))
    return pd.concat(dfs).reset_index()

df = isolines(xr, yr, z, [0.3, 1, 2, 4, 8, 16, 32])
print(df)

iso = alt.Chart(df).mark_line().encode(
    x=alt.X('x:Q'),
    y=alt.Y('y:Q'),
    order='order',
    color=alt.Color('iso:O', scale=alt.Scale(scheme='category10'))
).properties(
    width=200,
    height=200
)

contour+iso

      index         x         y  order   iso
0         0 -0.444031 -0.306122      0   0.3
1         1 -0.510204 -0.173776      1   0.3
2         2 -0.534116 -0.102041      2   0.3
3         3       NaN       NaN      3   0.3
4         4 -0.444031  0.306122      4   0.3
...     ...       ...       ...    ...   ...
1857    283  4.019490  3.979592   1857  32.0
1858    284       NaN       NaN   1858  32.0
1859    285  2.959184 -4.820663   1859  32.0
1860    286  2.974411 -4.811146   1860  32.0
1861    287       NaN       NaN   1861  32.0

[1862 rows x 5 columns]


alt.LayerChart(...)

In [7]:
from spinorama.load import graph_melt
from spinorama.load_parse import parse_all_speakers, parse_graphs_speaker
from spinorama.contour import compute_contour
from spinorama.graph import contour_params_default, graph_contour_common


df = parse_graphs_speaker('Klipsch', 'Klipsch R41M', 'klippel')
#df = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')

ImportError: cannot import name 'parse_all_speakers' from 'spinorama.load_parse' (/Volumes/Home/pierre/src/spinorama/src/spinorama/load_parse.py)

In [ ]:
dfu = df['SPL Horizontal_unmelted']
af, am, az = compute_contour(dfu)
contour = graph_contour_common(af, am, az, contour_params_default)
print(len(af[0]), len(am[:,0]), az.shape )



In [ ]:
df = isolines(af[0], am[:,0], az.T, [-12, -9, -6, -3,  -2, -1, 3])
# print(df)

iso = alt.Chart(df
).mark_line(
    clip=True,
    thickness=3,
    interpolate='basis',
).encode(
    x=alt.X('x:Q', scale=alt.Scale(type='log', domain=[20, 20000], nice=False)),
    y=alt.Y('y:Q', scale=alt.Scale(type='linear', domain=[-180, 180], nice=False)),
    order='order',
    color=alt.Color('iso:O', scale=alt.Scale(scheme='plasma'))
).properties(width=1000, height=800)

iso